# CS636 Final Project
### group member: Chinghao Sun, Tianqi Xu, Haotian Yin, Chen Ye

### Predict Future Sales
* https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data

Goals

1. We are not competing with other teams on kaggle
2. This project is a playground to practice the knowledge of this class and prepare for the final exam.
3. Group project, 1-4 people per team
4. You can use R or Python
5. Build one prediction model using the ML algorithms of this course
6. Evaluate your prediction model
7. Try different ways to improve your model and show the improvements.
8. Submit code and results in Jupyter and HTML formats on canvas

# Importing all the required libraries at once so that they can systematically be used when required

In [29]:
import numpy as np
import pandas as pd
import datetime 
import warnings
warnings.simplefilter("ignore")
# ignore all the warning messages but show the errors
sales_train = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Load all the necessary files for data

In [30]:
sales_train.head()
# shop_id with shop
sales_shop = sales_train.merge(shops, on = 'shop_id')
# item_id with items
sales_shop_items = sales_shop.merge(items, on = 'item_id')
sales_shop_items_itmcat = sales_shop_items.merge(item_categories, on ='item_category_id')
sales_train.shape

(2935849, 6)

In [3]:
test.head()
# item_id with items,sales_train
# ID with submission

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [4]:
submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [5]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [31]:
df= sales_shop_items_itmcat
df.shape

(2935849, 10)

In [32]:
sales_train.columns
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


## Preprocessing data

### Feature selection from data by analysis the problem
Based on the test data, the only features test data give is shop_id and item_id. 

In training data set, two more features are given. dat_block_num is the number of month. This information is included in the item of feature date. 

item_price is given in training data, but not given in test data. By looking at the training data, we find that the price of items decreases by date. This is just a simple model, and it is not quite straight to predict item_cnt by predicting price.So in this model, we do not consider the effect of price time series.

shop_id also indicates shop_name. Because the name is in Russian and we cannot judge the popularity of shop by its name, so we do not use shop name in our model.

In [10]:
sales_train.drop(['date_block_num','item_price'], axis=1, inplace=True)
sales_train['date'] = pd.to_datetime(sales_train['date'], dayfirst=True)
sales_train['date'] = sales_train['date'].apply(lambda x: x.strftime('%Y-%m'))
sales_train.columns

Index(['date', 'shop_id', 'item_id', 'item_cnt_day'], dtype='object')

In [33]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


### Grouping dataset “sales_train” by month.:   
Since prediction of October 2015 is asked, each month’s overall data will be needed.
### columns of “shop_id” and “item_id” are needed:    
to dataset “test”, in order to identify the forecasting for the work later.

In [11]:
df = sales_train.groupby(['date','shop_id','item_id']).sum()
df_train = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df_train.reset_index(inplace=True)
df_train.head()

date,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,1,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Applying model on data

Obviously, the data is a time series. What we want to do is predicting value of time k+1 by values from the first k time. 

In [12]:
df_test = pd.merge(test, df_train, on=['shop_id','item_id'], how='left')
df_test.drop(['ID', '2013-01'], axis=1, inplace=True)
df_test = df_test.fillna(0)
df_test.head()

,shop_id,item_id,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### We use MultiLinear Regression to deal with the time series

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn import neighbors, preprocessing, model_selection

In [14]:
y = df_train['2015-10'].values
X= df_train.drop(['2015-10'], axis = 1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### result and its evaluation

In [36]:
y_pred = regressor.predict(X_test)
y_pred

array([ -1.73507635,  -3.35614731, -21.39642121, ..., -16.68446187,
       -19.06247155, -10.70246695])

In [16]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared Error:', metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 0.5448161473564892
Mean Squared Error: 3.6181446328753046
Root Mean Squared Error: 1.9021421168975006
R-squared Error: 0.2641081693130485


### How to improve the model?  
First R-squared Error is  -0.5611     
How to improve the model?    
Multilinear Regression is an outlier sensitive model.    
Getting rid of outliers by z-score probable can be a solution.   

In [17]:
from sklearn.preprocessing import StandardScaler
df_train_z=df_train.drop(['shop_id','item_id'],axis=1)
df_train_z.head()
df  = StandardScaler().fit_transform(df_train_z.values)
cols = df_train_z.columns
df= pd.DataFrame(df,columns = cols)
df['shop_id']=df_train['shop_id']
df['item_id']=df_train['item_id']
df = df[df.values<=3]

y = df['2015-10'].values
X= df.drop(['2015-10'], axis = 1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm
y_pred = regressor.predict(X_test)
# y_pred
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared Error:', metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 0.06245299712268311
Mean Squared Error: 0.20838598224376584
Root Mean Squared Error: 0.4564931349360753
R-squared Error: 0.1702437301769999


However, the Filter method by Scaling works on the improvement  even is not incredibly efficient. 

## improvement attempt 2: Decision Tree

In [18]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
y = df_train['2015-10'].values
X= df_train.drop(['2015-10'], axis = 1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
dt.feature_importances_
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared Error:', metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 0.18993221338048924
Mean Squared Error: 4.949708222811671
Root Mean Squared Error: 2.224793973115639
R-squared Error: 0.04148360723555278


In [19]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
y = df['2015-10'].values
X= df.drop(['2015-10'], axis = 1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
dt.feature_importances_
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared Error:', metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 1.6834604071303355e-14
Mean Squared Error: 1.2691682347555644e-27
Root Mean Squared Error: 3.5625387503233764e-14
R-squared Error: 1.0


The improvement is more efficient on the DecisionTreeRegressor model.    
 DecisionTreeRegressor model improved from R square 0.1077 to 0.99.   
 The overfitting should be concerned on this high R square.

## improvement attempt 3: Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
df = sales_train.groupby(['date','shop_id','item_id']).sum()
df_train = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df_train.reset_index(inplace=True) 

RFR = RandomForestRegressor(n_estimators = 100)
y = df_train['2015-10'].values
X= df_train.drop(['2015-10'], axis = 1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3,random_state=101)
RFR.fit(X_train, y_train)
y_pred = dt.predict(X_test)
RFR.feature_importances_
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared Error:', metrics.r2_score(y_test, y_pred))

D:\software\AnacondaPython\setup\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Mean Absolute Error: 0.30781549224355104
Mean Squared Error: 6.823675851878197
Root Mean Squared Error: 2.6122166548504735
R-squared Error: -0.31594849769337574


We also think of adding item_category as another feature. However, the improvement in result is not quite obvious. So we stick to our attempt on the Decision Tree.

# Evaluation of Model
Because the huge differences of R-square of DecisionTreeRegressor on running  multiple times.    
Cross validation should be a better evaluation.

In [21]:
seed = 1
kfold = model_selection.KFold(n_splits=10)

model = dt
results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print('Accuracy -val set: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy -val set: 89.64% (0.02)
